<a href="https://colab.research.google.com/github/fjadidi2001/Image_Inpaint/blob/main/HINT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
from datasets import load_dataset
import pandas as pd

# Load the CelebA-HQ dataset
dataset = load_dataset("saitsharipov/CelebA-HQ")

# Check if dataset loaded correctly
if dataset is None:
    raise ValueError("Dataset not loaded correctly")


README.md:   0%|          | 0.00/224 [00:00<?, ?B/s]

(…)-00000-of-00003-30fbdf94a5d8e722.parquet:   0%|          | 0.00/464M [00:00<?, ?B/s]

(…)-00001-of-00003-74b418aa7a721747.parquet:   0%|          | 0.00/464M [00:00<?, ?B/s]

(…)-00002-of-00003-4f9459a488896f53.parquet:   0%|          | 0.00/465M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/202599 [00:00<?, ? examples/s]

In [ ]:

# Define image transformations (e.g., resizing, normalization)
transform = transforms.Compose([
    transforms.Resize((256, 256)),  # resize to match model input size
    transforms.ToTensor(),          # convert to tensor
])

# Apply transformations to the dataset
def transform_examples(example):
    example["image"] = transform(example["image"])
    return example

# Transform dataset with the custom function
dataset = dataset.map(transform_examples)

# Create DataLoader
batch_size = 4
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

Map:   0%|          | 0/202599 [00:00<?, ? examples/s]

TypeError: Unexpected type <class 'list'>

In [ ]:
from torchvision import transforms
from torch.utils.data import DataLoader
from PIL import Image


# Define image transformations (e.g., resizing, normalization)
transform = transforms.Compose([
    transforms.Resize((256, 256)),  # resize to match model input size
    transforms.ToTensor(),          # convert to tensor
])

# Apply transformations to the dataset
def transform_examples(example):
    # Ensure the image is in PIL format
    example["image"] = transform(Image.fromarray(example["image"]))
    return example

# Transform dataset with the custom function
dataset = dataset.map(transform_examples)

# Create DataLoader
batch_size = 4
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)


Map:   0%|          | 0/202599 [00:00<?, ? examples/s]

TypeError: Unexpected type <class 'list'>

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# Define Mask-aware Pixel-shuffle Downsampling (MPD)
class MaskAwarePixelShuffle(nn.Module):
    def __init__(self, scale_factor=2):
        super(MaskAwarePixelShuffle, self).__init__()
        self.scale_factor = scale_factor
        self.conv = nn.Conv2d(3, 3 * (scale_factor ** 2), kernel_size=3, padding=1)

    def forward(self, x, mask):
        x = self.conv(x)
        batch, channels, height, width = x.shape
        x = x.view(batch, channels // (self.scale_factor ** 2), height * self.scale_factor, width * self.scale_factor)
        mask = mask.view(batch, 1, height * self.scale_factor, width * self.scale_factor)
        return x, mask

# Define Spatially-activated Channel Attention Layer (SCAL)
class SCAL(nn.Module):
    def __init__(self, channels):
        super(SCAL, self).__init__()
        self.channel_attention = nn.Conv2d(channels, channels, kernel_size=1)
        self.spatial_attention = nn.Conv2d(channels, channels, kernel_size=3, padding=1)

    def forward(self, x):
        channel_attn = self.channel_attention(x)
        spatial_attn = self.spatial_attention(x)
        return channel_attn * spatial_attn

# Define the Sandwich Transformer Block
class SandwichBlock(nn.Module):
    def __init__(self, channels):
        super(SandwichBlock, self).__init__()
        self.ffn1 = nn.Conv2d(channels, channels, kernel_size=1)
        self.scal = SCAL(channels)
        self.ffn2 = nn.Conv2d(channels, channels, kernel_size=1)

    def forward(self, x):
        x = F.relu(self.ffn1(x))
        x = self.scal(x)
        return F.relu(self.ffn2(x))

# Define the HINT Model
class HINTModel(nn.Module):
    def __init__(self, in_channels=3, out_channels=3, base_channels=64):
        super(HINTModel, self).__init__()
        self.embed = nn.Conv2d(in_channels, base_channels, kernel_size=3, padding=1)
        self.sandwich_blocks = nn.ModuleList([SandwichBlock(base_channels) for _ in range(4)])
        self.output = nn.Conv2d(base_channels, out_channels, kernel_size=3, padding=1)

    def forward(self, x, mask):
        x = self.embed(x)
        for block in self.sandwich_blocks:
            x = block(x)
        return self.output(x)

# Instantiate the model
model = HINTModel()


In [ ]:
# Step 3: Define and Instantiate the Loss Function
class HINTLoss(nn.Module):
    def __init__(self, lambda_l1=1, lambda_style=250, lambda_perc=0.1, lambda_adv=0.001):
        super(HINTLoss, self).__init__()
        self.l1_loss = nn.L1Loss()
        self.lambda_l1 = lambda_l1
        self.lambda_style = lambda_style
        self.lambda_perc = lambda_perc
        self.lambda_adv = lambda_adv

    def forward(self, output, target):
        # L1 Loss
        l1_loss = self.l1_loss(output, target)

        # Style, perceptual, and adversarial losses would go here, if implemented
        # For simplicity, we’re only using L1 in this example.

        total_loss = self.lambda_l1 * l1_loss  # + other loss terms if implemented
        return total_loss

# Instantiate the loss function as 'criterion'
criterion = HINTLoss()

# Step 4: Define and Run the Training Loop
def train(model, dataloader, criterion, epochs=10, lr=1e-4):
    optimizer = optim.Adam(model.parameters(), lr=lr)

    for epoch in range(epochs):
        epoch_loss = 0
        for batch in dataloader:
            images = batch["image"]
            masks = torch.ones_like(images)  # Placeholder for actual mask input

            optimizer.zero_grad()
            outputs = model(images, masks)
            loss = criterion(outputs, images)  # Assuming inpainting target is original image
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()

        print(f"Epoch {epoch + 1}/{epochs}, Loss: {epoch_loss / len(dataloader)}")

# Run training
train(model, dataloader, criterion)


KeyError: "Invalid key: 0. Please first select a split. For example: `my_dataset_dictionary['train'][0]`. Available splits: ['train']"